In [1]:
from torch import tensor, stft, hann_window, abs, square
from torch.cuda import is_available
from time import time
from os import makedirs
from os.path import join

from utils_basic import SPECTROGRAM_DIR as outdir, GEO_STATIONS as stations
from utils_basic import get_geo_metadata, get_geophone_days
from utils_torch import get_hourly_geo_spectrograms_for_a_day
from utils_spec import save_geo_spectrograms
from utils_preproc import read_and_process_day_long_geo_waveforms

In [2]:
# Inputs
cuda = False
window_length = 1.0
overlap = 0.0
downsample = False
outdir = join(outdir, "hourly")

In [3]:
# Check if CUDA is available
if cuda == True:
    if is_available():
        print("GPU is available")
    else:
        print("GPU is NOT available")

In [4]:
# Load the list of days
#days = get_geophone_days()
days = ["2020-01-13"]

In [5]:
# Create the output directory
makedirs(outdir, exist_ok = True)

In [6]:
# Load the geophone metadata
metadat = get_geo_metadata()

In [7]:
# Loop over all days and stations
for day in days:
    for station in stations:
        print("######")
        print(f"Working on {day} for {station}...")
        print("######")
        
        clock1 = time()

        # Read and preprocess
        print(f"Reading and preprocessing the waveforms...")
        stream_day = read_and_process_day_long_geo_waveforms(day, metadat, stations = station)

        if stream_day == None:
            print("No data found! Skipped.")
            continue

        # Compute the hourly spectrograms
        print(f"Computing hourly spectrograms...")
        stream_spec, stream_spec_ds = get_hourly_geo_spectrograms_for_a_day(stream_day, 
                                                                            window_length = window_length, overlap = overlap, cuda = cuda, downsample = downsample)

        # Save the results     
        print("Saving the spectrograms...")
        time_labels = stream_spec.get_time_labels()
        for time_label in time_labels:
            stream_spec_hour = stream_spec.select(time_label = time_label)
            
            filename = f"hourly_geo_spectrograms_{time_label}_{station}_window{window_length:.0f}s_overlap{overlap:.1f}.h5"
            save_geo_spectrograms(stream_spec_hour, filename, outdir = outdir)
        
        clock2 = time()
        
        elapse_time = clock2 - clock1
        print(f"Elapsed time: {elapse_time:f} s")

######
Working on 2020-01-13 for A01...
######
Reading and preprocessing the waveforms...
Reading the waveforms for 2020-01-13
Preprocessing the waveforms...
Computing hourly spectrograms...
Computing the spectrograms for starttime 2020-01-13T00:00:00.000000Z...
Computing the spectrograms for starttime 2020-01-13T01:00:00.000000Z...
Computing the spectrograms for starttime 2020-01-13T02:00:00.000000Z...
Computing the spectrograms for starttime 2020-01-13T03:00:00.000000Z...
Computing the spectrograms for starttime 2020-01-13T04:00:00.000000Z...
Computing the spectrograms for starttime 2020-01-13T05:00:00.000000Z...
Computing the spectrograms for starttime 2020-01-13T06:00:00.000000Z...
Computing the spectrograms for starttime 2020-01-13T07:00:00.000000Z...
Computing the spectrograms for starttime 2020-01-13T08:00:00.000000Z...
Computing the spectrograms for starttime 2020-01-13T09:00:00.000000Z...
Computing the spectrograms for starttime 2020-01-13T10:00:00.000000Z...
Computing the spe